# Conversor de arquivos `.data` para `.xdi`

In [1]:
import regex as re
import pandas as pd
import os

## Objetivo

<font size=3>Com o intuito de adquirir mais dados no formato `.xdi`, criamos um script para converter os arquivos `.data` encontrados para o formato `.xdi`</font>

___
<font color='red' size=5>**Atenção**</font>

<font size=3>Antes de executar o script, certifique-se que a variável `caminho_raiz` e `caminho_pasta_convertidos` estão com o seu diretório.</font>
___

## Montando o arquivo Geral

<font size=3>Dada a quantidade de arquivos que temos, vamos criar uma rotina que identifique os arquivos e armazene os caminhos em uma lista para serem acessados posteriormente para conversão.</font>

In [6]:
#obtendo arquivo
# Caminho para a pasta raiz que você deseja verificar
caminho_raiz = 'coloque o seu diretório aqui'

# Lista para armazenar os nomes de todos os arquivos .data encontrados
arquivos_data = []

# Expressão regular para identificar arquivos .data
padrao_data = re.compile(r'(.+)?\.data.txt$')

#Criando lista para guardar os nomes das pastas dos dados convertidos
nomes_pastas = []

# Percorrendo recursivamente todos os diretórios e subdiretórios
for diretorio_atual, subdiretorios, arquivos in os.walk(caminho_raiz):
    # Iterando sobre os arquivos no diretório atual
    if not nomes_pastas:
        nomes_pastas.extend(subdiretorios)
    for nome_arquivo in arquivos:
        # Verificando se o arquivo corresponde ao padrão .data
        if padrao_data.search(nome_arquivo):
            # Construindo o caminho completo do arquivo
            caminho_completo = os.path.join(diretorio_atual, nome_arquivo)
            # Adicionando o caminho completo à lista de arquivos .data
            arquivos_data.append(caminho_completo)

# Separando os dados

In [2]:
def separador():
    '''Esta função identifica os dados no arquivo data e separa os tópicos presentes no arquivo.
    
    returns:
        retorna uma lista contendo apenas os metadados.
    '''
    lista_sentencas = [r'\*[S-s]pectrum\*\n', r'\*[S-s]ample\*\n', r'\*Experiment\*\n', r'\*Data reference\*\n']
    lista_titulos = []
    for sentenca in lista_sentencas:
        for linha in texto:
            if re.search(sentenca, linha) != None:
                lista_titulos.append(re.search(sentenca, linha))

    for titulo in lista_titulos:
        for linha in dados_header:
            if titulo[0] == linha:
                dados_header.remove(linha)

    dados_header2 = []
    for dados in dados_header:
        dados_header2.append(dados.replace('\n', ''))
        
    separados = []

    for linha in dados_header2:
        separados.append(re.split(r'\-?\s*(?=\w:)?\s*\:', linha)) #com ajuda do chatGPT
    
    return separados

In [8]:
tk_initial = '#'
tk_endline = '---------'
tk_comment = '////'

def monta_caso (nome, valor):
    '''Esta função recebe o metadado e um valor e adiciona o vlaor ao metadado xdi correspondente.
    Args:
        nome: metadado do arquivo xdi.
        valor: informação a ser armazenada no metadado.
    returns:
        retorna o par metadado e informação a ser inserido no documento xdi.
    '''
    
    match nome:
        case 'version':
            return f'{tk_initial} XDI/{valor}\n'
        
        case 'Column.1':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Column.2':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Element.symbol':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Element.edge':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Mono.name':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Mono.d_spacing':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Beamline.name':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Sample.name':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Sample.temperature':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Sample.pressure':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'comment':
            return f'{tk_initial} {tk_comment}\n{valor}\n'
        
        case 'end_line':
            return f'{tk_initial}{tk_endline}\n'
        
        case 'ini_df':
            return f'{tk_initial} {valor}\n'
        
        case 'data':
            return f'{tk_initial} {valor}'

In [9]:
def buscador():
    '''Esta função busca os valores armazenados nos metadados do arquivo data.
    returns:
        retorna uma string que une todos as informações que estavam nos metadados a serem encontrados.
        
    '''
    sentenca1 = r'-\s[N-n]ame:\s(.+)?' #- Name:
    sentenca2 = r'-\s[C-c]omments:\s(\s+)?'#- Comments:
    sentenca3 = r'-\s[T-t]emperature:\s(.+)?' #- Temperature:
    sentenca4 = r'-\s[P-p]ressure:\s(.+)?'#- Pressure:

    for linha in texto: #parte do comentario
        if linha == '*Data reference*\n':

            ind_autor = texto.index(linha)
            autores = texto[ind_autor+1]
            autores = '#' + autores

        if re.search(sentenca1, linha) != None:

            sample_name = re.search(sentenca1, linha)[1]

        if re.search(sentenca3, linha) != None:

            sample_temperature = re.search(sentenca3, linha)[1]
        
        if re.search(sentenca4, linha) != None:

            sample_pressure = re.search(sentenca4, linha)[1]
            
        if re.search(sentenca2, linha) != None:

            fala = re.search(sentenca2, linha)[1]
            if not isinstance(fala, str):
                fala = ' '
            
            fala = '#' + fala
    ref_sshade = "# Schmitt, Bernard; Bollard, Philippe; Albert, Damien; Garenne, Alexandre; Bonal, Lydie; and the SSHADE partner's consortium (2018). SSHADE: 'Solid Spectroscopy Hosting Architecture of Databases and Expertise'. OSUG Data Center. Service/Database infrastructure. https://doi.org/10.26302/SSHADE"
    return fala + autores + ref_sshade, sample_name, sample_temperature, sample_pressure

In [13]:
def estrutura_arquivo(colunas_df, busca, sample_name, sample_temperature, sample_pressure, comentarios, dados_df):
    '''Esta função monta um dicionário com o par metadado informação em XDI.
    Args:
        colunas_df: lista contendo os títulos do dataframe do espectro
        busca: lista contendo informações sobre o nome do experimento
        sample_name: nome da amostra do experimento
        sample_temperature: temperatura na qual o experimento foi realizado
        sample_pressure: pressão na qual o experimento foi realizado
        comentarios: comentários contidos no arquivo data
        dados_df: string contendo os dados de energia e transmissão/fluerescência do espectro medido
        
    returns:
        retorna um dicionário contendo a estrutura do arquivo XDI que será montado.
        '''
    estrutura_arq = {}
    for elemento in busca:
        if re.search(r'(.+?)-(.+?)\d\,\d', elemento) != None:
            busca[1]= elemento[0]
            busca[2] = elemento

    estrutura = ['version', 'Column.1', 'Column.2','Element.symbol', 'Element.edge', 'Mono.name', 'Mono.d_spacing', 'Beamline.name', 'Sample.name', 'Sample.temperature', 'Sample.pressure', 'comment', 'end_line', 'data']
    valor = [1.0, colunas_df[0], colunas_df[1], busca[1], busca[2], 'Si 2 2 0', 1.920, 'FAME', sample_name, sample_temperature, sample_pressure, comentario, 0, dados_df]

    for item, val in zip (estrutura, valor):

        estrutura_arq[item] = monta_caso(item, val)
    
    return estrutura_arq

In [14]:
def monta_arquivo(estrutura_arq, caminho_arquivo_convertido):
    '''Esta função monta o arquivo XDI.
    Args:
        estrutura_arq: dicionário contendo as estruturas do arquivo XDI.
        caminho_arquivo_convertido: caminho do arquivo novo XDI que será gerado.'''
    conteudo=''
    formato = '.xdi'
    lista_erros = []
    for values in estrutura_arq.values():
        conteudo += values
    
        with open(caminho_arquivo_convertido+formato, 'w') as arqu:
            arqu.write(conteudo)

In [15]:
#estruturando o código completo
caminho_pasta_convertidos = 'coloque o seu diretório aqui'
lista_arquivos_erro = []
lista_arquivos_naoXAS = []
i = 0
for arquivo in arquivos_data:
    texto = ''
    print(f'-------\nConvertendo arquivo { arquivo}\n')
    # Criando a pasta para os arquivos convertidos, se ainda não existir
    if not os.path.exists(caminho_pasta_convertidos):
        os.mkdir(caminho_pasta_convertidos)
    try:
        arqui = open(arquivo, 'r')
        texto = arqui.readlines()
        #with open(arquivo, 'r') as arqui: #usar realine(s)
        #    texto = arqui.readlines()
        padrao = re.compile(r'energy\(ev\)\t.+')

        labels = 0
        for linha in texto:
            if padrao.search(linha) != None:
                labels = linha

        if labels == 0:
            print(f'O arquivo {arquivo} não possui dataframe XAS em seu formato.')
            lista_arquivos_naoXAS.append(arquivo)
            arquivos_data.remove(arquivo)

        else:
            dados_header = texto[:texto.index(labels)]
            dados_dataframe = texto[texto.index(labels):]

            cols_splt = dados_dataframe[0].replace('\n', "")
            colunas_df = cols_splt.split('\t')

            separados = separador()

            sentenca = r'(.+?)(\sedge|\sXES)' #montei com ajuda do GPT
            busca = re.search(sentenca, separados[0][1])
            print(f'Esse é o busca: {busca}')
            busca = busca[0].split(" ")
            dados_df = ''
            for item in dados_dataframe:
                dados_df+= item

            comentario, sample_name, sample_temperature, sample_pressure = buscador()

            estrutura_arq = estrutura_arquivo(colunas_df, busca, sample_name,sample_temperature, sample_pressure, comentario, dados_df)
            # Criando a subpasta com o nome do elemento
            nome_elemento = busca[1]
            caminho_pasta_elemento = os.path.join(caminho_pasta_convertidos, nome_elemento)

            if not os.path.exists(caminho_pasta_elemento):
                os.mkdir(caminho_pasta_elemento)

            # Montando o caminho completo para o arquivo convertido
            nome = separados[0][1]
            nome = nome.replace('/', '_')
            caminho_arquivo_convertido = os.path.join(caminho_pasta_elemento,nome)
            print(f'Esse é o caminho da Luz: {caminho_arquivo_convertido}\n\n')
            monta_arquivo(estrutura_arq, caminho_arquivo_convertido)
            print(f'\nArquivo convertido em {caminho_arquivo_convertido}\n----------\n\n')
            
    except FileNotFoundError:
        lista_arquivos_erro.append(caminho_do_arquivo)
    i += 1
print(f'Arquivos não convertidos: {lista_arquivos_erro}')
print(f'Arquivos não XAS: {lista_arquivos_naoXAS}')
print(f'Os {i} arquivos foram convertidos com sucesso!')

-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Ag\XAS_Ag-GSH_1-1\XAS_Ag-GSH_1-1.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' Ag K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS fluorescence Ag-GSH 1



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS fluorescence Ag-GSH 1
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Ag\XAS_AgL1_1-1\XAS_AgL1_1-1.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' Ag K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS fluorescence of AgL1 complex solution (L1=NTA(CysOEt)3 and pH=7.4) at 16K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS fluorescence of AgL1 complex solution (L1=NTA(CysOEt)3 and pH=7.4) at 16K
----------


Esse é o busca: <regex.Match object; span=(0, 10), match=' Ag K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS transmission of Ag-diethyldithiocarbamate at 10K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS transmission of Ag-diethyldithiocarbamate at 10K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Ag\XAS_trans_PVPAgNP_10K\XAS_trans_PVPAgNP_10K.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' Ag K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS transmission of PVP-coated Ag nano-particles at 10K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS transmission of PVP-coated Ag nano-particles at 10K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Au\XAS_trans_


Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Br\ Br K edge XAS transmission of gallium bromide solution (Ga3+ 0.17m, Br- 0.51m) at 300bar and 470K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Br\XAS_Brtrans_GaBr3_0.17m_300b_560K\XAS_Brtrans_GaBr3_0.17m_300b_560K.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' Br K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Br\ Br K edge XAS transmission of gallium bromide solution (Ga3+ 0.17m, Br- 0.51m) at 300bar and 560K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Br\ Br K edge XAS transmission of gallium bromide solution (Ga3+ 0.17m, Br- 0.51m) at 300bar and 560K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Br\XAS_Brtrans_GaBr3_0.17m_300b_605K\XAS_Brtrans_GaBr3_0.17m_300b_605K.data.txt

Esse é o busca: <regex.Match

Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Cd\ Cd K edge XAS fluorescence of Cd-citrate at 15K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Cd\ Cd K edge XAS fluorescence of Cd-citrate at 15K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Cd\XAS_fluo_Cd-cysteine_15K\XAS_fluo_Cd-cysteine_15K.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' Cd K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Cd\ Cd K edge XAS fluorescence of Cd-cysteine at 15K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Cd\ Cd K edge XAS fluorescence of Cd-cysteine at 15K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Cd\XAS_fluo_Cd-GSH_15K\XAS_fluo_Cd-GSH_15K.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' Cd K edge'>
Esse é o caminho d


Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ce\ Ce L3 edge XAS HERFD of CeO2 (nanos) at 10K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Ce\XAS_CAS_LungCeO2_10K\XAS_CAS_LungCeO2_10K.data.txt

Esse é o busca: <regex.Match object; span=(0, 11), match=' Ce L3 edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ce\ Ce L3 edge XAS HERFD of mouse lung exposed to CeO2 nanoparticles at 10K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ce\ Ce L3 edge XAS HERFD of mouse lung exposed to CeO2 nanoparticles at 10K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Co\XAS_fluo_CoAl2O3\XAS_fluo_CoAl2O3.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' Co K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Co\ Co K edge XAS fluorescence of Co doped 

...


Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\V\ V K edge XAS HERFD of Pellet of V2O3 diluted with BN at 2500 ppm of V
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\V\V_K\XAS_HERFD_V2O4_10K\XAS_HERFD_V2O4_10K.data.txt

Esse é o busca: <regex.Match object; span=(0, 9), match=' V K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\V\ V K edge XAS HERFD of Pellet of V2O4 diluted with BN at 2400 ppm of V



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\V\ V K edge XAS HERFD of Pellet of V2O4 diluted with BN at 2400 ppm of V
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\V\V_K\XAS_HERFD_V2O5_10K\XAS_HERFD_V2O5_10K.data.txt

Esse é o busca: <regex.Match object; span=(0, 9), match=' V K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\V\ V K edge XAS HERFD o

Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\W\ W L3 edge XAS transmission of Na2WO4 sodium tungstate pellet at 10K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\W\ W L3 edge XAS transmission of Na2WO4 sodium tungstate pellet at 10K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\W\XAS_trans_Na2WO4_Solution_10K\XAS_trans_Na2WO4_Solution_10K.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' W L3 edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\W\ W L3 edge XAS transmission of Na2WO4 sodium tungstate aqueous solution at 10K



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\W\ W L3 edge XAS transmission of Na2WO4 sodium tungstate aqueous solution at 10K
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\W\XAS_trans_W_300K\XAS_trans_W_300K.dat

Esse é o busca: <regex.Match object; span=(0, 9), match=' Y K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.011m Cl 0.79m HCl 0.75m) at 800 bar and 200Â°C



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.011m Cl 0.79m HCl 0.75m) at 800 bar and 200Â°C
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Y\XAS_fluo_DT_20170222_003_30\XAS_fluo_DT_20170222_003_30.data.txt

Esse é o busca: <regex.Match object; span=(0, 9), match=' Y K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.011m Cl 0.79m HCl 0.75m) at 800 bar and 30Â°C



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\

Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.92m Cl 2.76m) at 800 bar and 400Â°C



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.92m Cl 2.76m) at 800 bar and 400Â°C
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Y\XAS_fluo_DT_20170222_005_450\XAS_fluo_DT_20170222_005_450.data.txt

Esse é o busca: <regex.Match object; span=(0, 9), match=' Y K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.92m Cl 2.76m) at 800 bar and 450Â°C



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions


Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Zr\ Zr K edge XAS fluorescence of synthetic baddeleyite ZrO2 at ambient conditions
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Zr\XAS_fluo_eudialyte\XAS_fluo_eudialyte.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), match=' Zr K edge'>
Esse é o caminho da Luz: C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Zr\ Zr K edge XAS fluorescence of natural eudialyte Na4(Ca,Ce,Fe,Mn,Y,La,Sr,K)3ZrSi8O22(OH,Cl)2 at ambient conditions



Arquivo convertido em C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Zr\ Zr K edge XAS fluorescence of natural eudialyte Na4(Ca,Ce,Fe,Mn,Y,La,Sr,K)3ZrSi8O22(OH,Cl)2 at ambient conditions
----------


-------
Convertendo arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\nao_convertidos\Zr\XAS_fluo_haplograniteZrFglass\XAS_fluo_haplograniteZrFglass.data.txt

Esse é o busca: <regex.Match object; span=(0, 10), ma

# Playground

<font size=4 color='red'>**Atenção**</font>
<font size

<font size=4>Esta seção é apenas o modo como a função de conversão acima foi construída, de modo a explicitar como o autor deste notebook montou a linha de raciocínio até chegar à função final.</font>

## Identificando os componentes do arquivo

In [2]:
arquivo = 'XAS_fluo_Y-doped_hematite_20K.data.txt'

with open(arquivo, 'r') as arqui: #usar realine(s)
    texto = arqui.readlines()

In [3]:
dados_header = texto[:texto.index('energy(ev)\tfluorescence_emission(au)\n')]
dados_dataframe = texto[texto.index('energy(ev)\tfluorescence_emission(au)\n'):]

## Separando Dados do Header

In [4]:
#Aplicando Regex para identificar as sentenças que armazenam o header do xdi (Spectrum, Sample)
#Como as sentenças são todas identificadas por *nome_sentenca*

In [5]:
#tentar melhorar esse código depois
def separador():
    lista_sentencas = [r'\*[S-s]pectrum\*\n', r'\*[S-s]ample\*\n', r'\*Experiment\*\n', r'\*Data reference\*\n']
    lista_titulos = []
    for sentenca in lista_sentencas:
        for linha in texto:
            if re.search(sentenca, linha) != None:
                lista_titulos.append(re.search(sentenca, linha))

    for titulo in lista_titulos:
        for linha in dados_header:
            if titulo[0] == linha:
                dados_header.remove(linha)

    dados_header2 = []
    for dados in dados_header:
        dados_header2.append(dados.replace('\n', ''))
        
    separados = []

    for linha in dados_header2:
        separados.append(re.split(r'\-?\s*(?=\w:)?\s*\:', linha)) #com ajuda do chatGPT
    
    return separados

In [17]:
#como separar cada informação e Pegar somente o que diz respeito ao *Spectrum*? Delimitando o espaço
#que há entre o fim de um título e começo de outro
#posso transformar cada título em um dicionário... (Chave = tudo antes do ':' e valor: tudo depois do ':' )

#regex para as linhas chave:valor
separados = separador()

#for linha in dados_header2:
    #separados.append(re.split(r'\-?\s*(?=\w:)?\s*\:', linha)) #com ajuda do chatGPT

## Separando Dados e Criando Dataframe

In [18]:
dados_df =[]
cols_splt = dados_dataframe[0].replace('\n', "")
colunas_df = cols_splt.split('\t')
dd_ssplt = []
for i in range(1, len(dados_dataframe)):
    dado = dados_dataframe[i].replace('\n', '')
    dd_ssplt.append(dado)
    dado_splt= dado.split('\t')
    dados_df.append(dado_splt)

In [19]:
colunas_df[0]

'energy(ev)'

In [20]:
dataframe = pd.DataFrame(dados_df, columns = colunas_df) #criando dataframe com pandas

In [21]:
#convertendo para float
dataframe['energy(ev)'] = dataframe['energy(ev)'].astype(float)
dataframe['fluorescence_emission(au)'] = dataframe['fluorescence_emission(au)'].astype(float)

## Testanto se o DataFrame produzido é fiel aos dados do arquivo

In [22]:
arquivo_teste = pd.read_csv('dados.txt', header=0, sep='\t')

In [23]:
dataframe.equals(arquivo_teste)

True

Isso confirma que os dados dos dois dataframes estão iguais! Isso é excelente, pois comprova que o algoritmo que cria os dados está funcionando de forma correta.

## Montando Meu ".xdi" 

In [24]:
separados

[['- Title', ' Y K edge XAS fluorescence of Y-doped hematite at 20K'],
 ['- Version', ' #1 (2023-11-10 10', '52', '59 UTC)'],
 ['- Measurement type', ' laboratory measurement (spectrum)'],
 ['- Spectrum type', ' fluorescence emission (AU)'],
 ['- Spectral unit', ' Energy (eV)'],
 ['- Valid spectral range(s)', ' 16850.0-17580.0 eV (Resol. 0.874 eV)'],
 ['- Observation geometry', ' directional'],
 ['- Spatial observation mode', ' averaged'],
 ['- Data analysis', ' raw averaged spectra'],
 ['- Quality flag', ' '],
 ['- File name', ' XAS_fluo_Y-doped_hematite_20K.data.txt'],
 ['- Comments', ' '],
 [''],
 ['- Name', ' Pellet of Y-doped hematite mixed with PVP'],
 ['- Origin(s)', ' laboratory; commercial'],
 ['- Type', ' pellet'],
 ['- Temperature', ' 20.0 +/- 278.15 K'],
 ['- Pressure', ' '],
 [''],
 ['- Title',
  ' Y K edge XAS transmission and XAS fluorescence of Yttrium components at 20K'],
 ['- Version', ' #1 (2023-11-10 10', '52', '59 UTC)'],
 ['- Experimentalist(s)',
  ' Couturier Jul

In [25]:
sentenca = r'(.+?)\sedge' #montei com ajuda do chat GPT
busca = re.search(sentenca, separados[0][1])

In [26]:
separados[0][1]

' Y K edge XAS fluorescence of Y-doped hematite at 20K'

In [27]:
busca[0][1] #te dá o elemento
busca[0][3] #te dá a borda de medida

'K'

In [28]:
tk_initial = '#'
tk_endline = '---------'
tk_comment = '////'

def monta_caso (nome, valor):
    
    match nome:
        case 'version':
            return f'{tk_initial} XDI/{valor}\n'
        
        case 'Column.1':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Column.2':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Element.symbol':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Element.edge':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Mono.name':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Mono.d_spacing':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Beamline.name':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'Sample.name':
            return f'{tk_initial} {nome}: {valor}\n'
        
        case 'comment':
            return f'{tk_initial} {tk_comment}\n{valor}\n'
        
        case 'end_line':
            return f'{tk_initial}{tk_endline}\n'
        
        case 'ini_df':
            return f'{tk_initial} {valor}\n'
        
        case 'data':
            return f'{tk_initial} {valor}'

In [29]:
dados_df = ''
for item in dados_dataframe:
    dados_df+= item

In [30]:
sentenca1 = r'-\s[N-n]ame:\s(.+)?'
sentenca2 = r'-\s[C-c]omments:\s(\s+)?'#- Comments:

for linha in texto: #parte do comentario
    if linha == '*Data reference*\n':
        
        ind_autor = texto.index(linha)
        autores = texto[ind_autor+1]
        
    if re.search(sentenca1, linha) != None:
        
        sample_name = re.search(sentenca1, linha)[1]
        
    if re.search(sentenca2, linha) != None:
        
        fala = re.search(sentenca2, linha)[1]

ref_sshade = "# Schmitt, Bernard; Bollard, Philippe; Albert, Damien; Garenne, Alexandre; Bonal, Lydie; and the SSHADE partner's consortium (2018). SSHADE: 'Solid Spectroscopy Hosting Architecture of Databases and Expertise'. OSUG Data Center. Service/Database infrastructure. https://doi.org/10.26302/SSHADE"

In [306]:
fala = '#' + fala

In [307]:
autores = '# ' + autores

In [308]:
comentario = autores + fala + ref_sshade

In [309]:
comentario

"# - Couturier, Julien; Lallemand, Claire; Levard, ClÃ©ment; Chaurand, Perrine; Rose, JÃ©rÃ´me; Collin, Blanche (2022): Y K edge XAS transmission and XAS fluorescence of Yttrium components at 20K. SSHADE/FAME (OSUG Data Center). Dataset/Spectral Data. https://doi.org/10.26302/SSHADE/EXPERIMENT_JC_20220615_001\n#\n# Schmitt, Bernard; Bollard, Philippe; Albert, Damien; Garenne, Alexandre; Bonal, Lydie; and the SSHADE partner's consortium (2018). SSHADE: 'Solid Spectroscopy Hosting Architecture of Databases and Expertise'. OSUG Data Center. Service/Database infrastructure. https://doi.org/10.26302/SSHADE"

In [310]:
estrutura_arq = {}

estrutura = ['version', 'Column.1', 'Column.2','Element.symbol', 'Element.edge', 'Mono.name', 'Mono.d_spacing', 'Beamline.name', 'Sample.name', 'comment', 'end_line', 'data']
valor = [1.0, colunas_df[0], colunas_df[1], busca[0][1], busca[0][3], 'Si 2 2 0', 1.920, 'FAME', sample_name, comentario, 0, dados_df]

for item, val in zip (estrutura, valor):
    
    estrutura_arq[item] = monta_caso(item, val)

In [313]:
conteudo=''
arquivo = f'.xdi'
for values in estrutura_arq.values():
    
    conteudo += values


with open(arquivo, 'w') as arqu:
    arqu.write(conteudo)